In [3]:
import cv2, glob, json, os
import numpy as np

# Adjust if you used a different board:
PATTERN_SIZE = (9, 6)        # inner corners
SQUARE_SIZE_MM = 25.0        # real square size (mm)
IMAGES_GLOB = "calib_images/*.jpg"
OUT_NPZ = "camera_params.npz"
OUT_REPORT = "camera_report.json"

TERM_CRIT = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
subpix_win = (11, 11)

def per_image_error(objp, imgp, rvec, tvec, K, D):
    proj, _ = cv2.projectPoints(objp, rvec, tvec, K, D)
    return float(np.sqrt(np.mean((proj.squeeze() - imgp.squeeze())**2)))

def main():
    imgs = sorted(glob.glob(IMAGES_GLOB))
    if not imgs:
        raise SystemExit("No images found in calib_images/. Run calib_capture.py first.")

    # 3D object points for a single view (Z=0 plane)
    objp = np.zeros((PATTERN_SIZE[0]*PATTERN_SIZE[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:PATTERN_SIZE[0], 0:PATTERN_SIZE[1]].T.reshape(-1, 2)
    objp *= (SQUARE_SIZE_MM)  # scale to mm (optional; not required for intrinsics, but useful for consistency)

    objpoints = []  # 3D points
    imgpoints = []  # 2D points
    img_size = None
    used = 0

    for f in imgs:
        img = cv2.imread(f)
        if img is None: 
            continue
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img_size is None: img_size = gray.shape[::-1]
        found, corners = cv2.findChessboardCorners(gray, PATTERN_SIZE,
                            flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if not found:
            print(f"[SKIP] Corners not found: {f}")
            continue
        corners_refined = cv2.cornerSubPix(gray, corners, subpix_win, (-1,-1), TERM_CRIT)
        objpoints.append(objp.copy())
        imgpoints.append(corners_refined)
        used += 1

    if used < 8:
        raise SystemExit(f"Not enough valid images ({used}). Capture at least 12–20 good views.")

    ret, K, D, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, img_size, None, None,
        flags=cv2.CALIB_RATIONAL_MODEL
    )

    # Per-image reprojection error
    per_view_err = []
    for i in range(len(objpoints)):
        err = per_image_error(objpoints[i], imgpoints[i], rvecs[i], tvecs[i], K, D)
        per_view_err.append(err)
    mean_err = float(np.mean(per_view_err))

    np.savez(OUT_NPZ, camera_matrix=K, dist_coeffs=D, image_size=np.array(img_size),
             rms=float(ret), mean_err=mean_err, per_view_err=np.array(per_view_err))
    report = {
        "image_size": {"width": img_size[0], "height": img_size[1]},
        "rms": float(ret),
        "mean_reprojection_error_px": mean_err,
        "num_used_images": used
    }
    with open(OUT_REPORT, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)
    print(f"[OK] Saved {OUT_NPZ}")
    print(f"[OK] RMS={ret:.4f} | mean reprojection error={mean_err:.3f} px")
    print(f"[INFO] Detailed report: {OUT_REPORT}")

if __name__ == "__main__":
    main()


SystemExit: No images found in calib_images/. Run calib_capture.py first.